In [4]:
!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [75]:
import pandas as pd
import sqlalchemy as db
import pymysql
import numpy as np

database_username='root' # Nombre de cliente en MySQL Workbrench
database_password='138012' # Contraseña de MySQL Workbrench
database_ip='localhost'
database_name='proyecto_03' # Nombre de Base de datos a donde nos conectaremos
database_conection=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
conexion=database_conection.connect()
metadata=db.MetaData()

In [76]:
df_accidentes = pd.read_csv('AccidentesAviones.csv')
df_accidentes

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,5003,"March 28, 2021",1835,"Near Butte, Alaska",Soloy Helicopters,?,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...
5004,5004,"May 21, 2021",1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi..."
5005,5005,"June 10, 2021",0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...
5006,5006,"July 04, 2021",11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,?,?,3,"While attempting to land at Jolo Airport, the ..."


In [77]:
df_accidentes = df_accidentes.drop(['Unnamed: 0'], axis=1)

In [78]:
df_accidentes.columns

Index(['fecha', 'HORA declarada', 'Ruta', 'OperadOR', 'flight_no', 'route',
       'ac_type', 'registration', 'cn_ln', 'all_aboard', 'PASAJEROS A BORDO',
       'crew_aboard', 'cantidad de fallecidos', 'passenger_fatalities',
       'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [79]:
df_accidentes = df_accidentes.rename(columns= {'fecha': 'Fecha siniestro', 'HORA declarada': 'Hora siniestro', 'Ruta':'Ruta aerea', 'OperadOR': 'Explotador aereo', 
                                                'flight_no': 'Numero de vuelo', 'route': 'Itinerario', 'ac_type' : 'Tipo de aeronave', 'registration': 'Matricula', 
                                                'cn_ln': 'num_ser_fabr_aeronave', 'all_aboard': 'Personas abordo', 'PASAJEROS A BORDO': 'Pasajeros a bordo', 'crew_aboard': 'Tripulacion a bordo',
                                                'passenger_fatalities' : 'Cant. pasajeros fallecidos', 'crew_fatalities': 'Cant. tripulacion fallecida', 'ground': 'Cant. personas en tierra fallecidas', 'summary': 'Resumen accidente',
                                                'cantidad de fallecidos': 'Cantidad de fallecidos' })

In [80]:
df_accidentes.replace(to_replace='?', value=np.nan, inplace=True)

In [81]:
df_accidentes.isnull().sum()

Fecha siniestro                           0
Hora siniestro                         1504
Ruta aerea                                5
Explotador aereo                         10
Numero de vuelo                        3682
Itinerario                              762
Tipo de aeronave                         13
Matricula                               272
num_ser_fabr_aeronave                   667
Personas abordo                          17
Pasajeros a bordo                       221
Tripulacion a bordo                     219
Cantidad de fallecidos                    8
Cant. pasajeros fallecidos              235
Cant. tripulacion fallecida             235
Cant. personas en tierra fallecidas      44
Resumen accidente                        59
dtype: int64

In [86]:
df_accidentes = df_accidentes.set_index(pd.DatetimeIndex(df_accidentes['Fecha siniestro']))
df_accidentes.sort_index(inplace=False)
df_accidentes = df_accidentes.drop(['Fecha siniestro'], axis=1)


In [87]:
df_accidentes

,Hora siniestro,Ruta aerea,Explotador aereo,Numero de vuelo,Itinerario,Tipo de aeronave,Matricula,num_ser_fabr_aeronave,Personas abordo,Pasajeros a bordo,Tripulacion a bordo,Cantidad de fallecidos,Cant. pasajeros fallecidos,Cant. tripulacion fallecida,Cant. personas en tierra fallecidas,Resumen accidente
Fecha siniestro,,,,,,,,,,,,,,,,
1908-09-17,1718,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
1912-07-12,0630,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
1913-08-06,NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
1913-09-09,1830,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20,NaN,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-28,1835,"Near Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...
2021-05-21,1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi..."
2021-06-10,0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...


In [88]:
df_accidentes['Cant. personas en tierra fallecidas'] = df_accidentes['Cant. personas en tierra fallecidas'].apply(pd.to_numeric)

In [89]:
df_accidentes.isnull().sum()

Hora siniestro                         1504
Ruta aerea                                5
Explotador aereo                         10
Numero de vuelo                        3682
Itinerario                              762
Tipo de aeronave                         13
Matricula                               272
num_ser_fabr_aeronave                   667
Personas abordo                          17
Pasajeros a bordo                       221
Tripulacion a bordo                     219
Cantidad de fallecidos                    8
Cant. pasajeros fallecidos              235
Cant. tripulacion fallecida             235
Cant. personas en tierra fallecidas      44
Resumen accidente                        59
dtype: int64

In [96]:
#pasar a mysql
df_accidentes.to_sql(name='accidentes', con=conexion)

In [103]:
df_acc = pd.read_csv('data.csv')
df_acc

,Unnamed: 0,Unnamed: 0.1,Date:,Time:,Location:,Operator:,Flight #:,Route:,AC Type:,Registration:,cn / ln:,Aboard:,Fatalities:,Ground:,Summary:
0,0,ACCIDENT DETAILS,"February 03, 1921",?,"Mendotta, Minnisota",US Aerial Mail Service,?,?,De Havilland DH-4,130,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,Shortly after takeoff from Minneapolis-World C...
1,1,ACCIDENT DETAILS,"February 09, 1921",?,"La Crosse, Wisconsin",US Aerial Mail Service,?,?,Junkers F-13,301,?,3 (passengers:0 crew:3),3 (passengers:0 crew:3),0,Crashed for unknown reasons. Both pilots and t...
2,2,ACCIDENT DETAILS,"February 15, 1921",?,Off Gibraltar,Aeropostale,?,?,Breguet 14,F-ALBA,?,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0,The mail fligh encountered poor weather condit...
3,3,ACCIDENT DETAILS,"February 22, 1921",?,"Elko, Nevada",US Aerial Mail Service,?,?,De Havilland DH-4,67,?,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,"Shortly after taking off, the aircraft stalled..."
4,4,ACCIDENT DETAILS,"April 06, 1921",?,"Point Cook, Australia",Military - Royal Australian Air Force,?,?,Avro 504,H3021,?,2 (passengers:0 crew:0),2 (passengers:0 crew:0),0,"Shortly after taking off on a training flight,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,11,ACCIDENT DETAILS,"May 19, 2016",0249,175 miles off the Egyptian coast,EgyptAir,804,Paris - Cairo,Airbus A320-232,SU-GCC,2088,66 (passengers:56 crew:10),66 (passengers:56 crew:10),0,"The plane was en route from Paris to Cairo, fl..."
5682,12,ACCIDENT DETAILS,"July 22, 2016",?,"Bay of Bengal, Indian Ocean",Indian Air Force,?,Chennai (Tambram AFB) - Port Blair,Antonov 32,K2743,0809,29 (passengers:23 crew:6),29 (passengers:23 crew:6),0,The aircraft was taking part in fire fighting ...
5683,13,ACCIDENT DETAILS,"August 05, 2016",1238,"Dubai, United Arab Emirates",Emirates,521,"Thiruvanthapuram, India - Dubai, UAE",Boeing 777-31H,A6-EMW,32700/434,300 (passengers:282 crew:18),0 (passengers:0 crew:0),1,"After being cleared to land, the crew informed..."
5684,14,ACCIDENT DETAILS,"August 31, 2016",1100,"Russian Mission, Alaksa",Hageland Aviiation,3190,Russian Mission- Marshall,Cessna 208 Grand Caravan,N752RV,208B5088,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0,A midair collision occurred between a Cessna 2...


In [104]:
df_acc = df_acc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [106]:
df_acc = df_acc.rename(columns= {'Date:': 'Fecha siniestro' , 'Time:': 'Hora siniestro', 'Location:': 'Ubicacion', 'Operator:' : 'Explotador aereo', 'Flight #:': 'Numero de vuelo', 
                                'Route:': 'Itinerario', 'AC  Type:': 'Tipo de aeronave', 'Registration:': 'Matricula' , 'cn / ln:' : 'num_ser_fabr_aeronave', 'Aboard:' : 'Personas a bordo', 'Fatalities:': 'Personas fallecidas', 
                                'Ground:': 'Cant. personas en tierra fallecidas ', 'Summary:': 'Resumen accidente'})

In [107]:
df_acc = df_acc.set_index(pd.DatetimeIndex(df_acc['Fecha siniestro']))
df_acc.sort_index(inplace=False)
df_acc = df_acc.drop(['Fecha siniestro'], axis=1)

In [108]:
df_acc.replace(to_replace='?', value=np.nan, inplace=True)

In [109]:
df_acc.to_csv('accidentes_2.csv')

In [110]:
a = pd.read_csv('accidentes_2.csv')
a

,Fecha siniestro,Hora siniestro,Ubicacion,Explotador aereo,Numero de vuelo,Itinerario,Tipo de aeronave,Matricula,num_ser_fabr_aeronave,Personas a bordo,Personas fallecidas,Cant. personas en tierra fallecidas,Resumen accidente
0,1921-02-03,NaN,"Mendotta, Minnisota",US Aerial Mail Service,NaN,NaN,De Havilland DH-4,130,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0.0,Shortly after takeoff from Minneapolis-World C...
1,1921-02-09,NaN,"La Crosse, Wisconsin",US Aerial Mail Service,NaN,NaN,Junkers F-13,301,NaN,3 (passengers:0 crew:3),3 (passengers:0 crew:3),0.0,Crashed for unknown reasons. Both pilots and t...
2,1921-02-15,NaN,Off Gibraltar,Aeropostale,NaN,NaN,Breguet 14,F-ALBA,NaN,2 (passengers:0 crew:2),2 (passengers:0 crew:2),0.0,The mail fligh encountered poor weather condit...
3,1921-02-22,NaN,"Elko, Nevada",US Aerial Mail Service,NaN,NaN,De Havilland DH-4,67,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0.0,"Shortly after taking off, the aircraft stalled..."
4,1921-04-06,NaN,"Point Cook, Australia",Military - Royal Australian Air Force,NaN,NaN,Avro 504,H3021,NaN,2 (passengers:0 crew:0),2 (passengers:0 crew:0),0.0,"Shortly after taking off on a training flight,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681,2016-05-19,0249,175 miles off the Egyptian coast,EgyptAir,804,Paris - Cairo,Airbus A320-232,SU-GCC,2088,66 (passengers:56 crew:10),66 (passengers:56 crew:10),0.0,"The plane was en route from Paris to Cairo, fl..."
5682,2016-07-22,NaN,"Bay of Bengal, Indian Ocean",Indian Air Force,NaN,Chennai (Tambram AFB) - Port Blair,Antonov 32,K2743,0809,29 (passengers:23 crew:6),29 (passengers:23 crew:6),0.0,The aircraft was taking part in fire fighting ...
5683,2016-08-05,1238,"Dubai, United Arab Emirates",Emirates,521,"Thiruvanthapuram, India - Dubai, UAE",Boeing 777-31H,A6-EMW,32700/434,300 (passengers:282 crew:18),0 (passengers:0 crew:0),1.0,"After being cleared to land, the crew informed..."
5684,2016-08-31,1100,"Russian Mission, Alaksa",Hageland Aviiation,3190,Russian Mission- Marshall,Cessna 208 Grand Caravan,N752RV,208B5088,5 (passengers:4 crew:1),5 (passengers:4 crew:1),0.0,A midair collision occurred between a Cessna 2...
